## Query 5

> Calculen el porcentaje de productos cuyo stock es al menos 20% más alto que el stock promedio de su marca. Por ejemplo, si el stock promedio de la marca Adidas fuera 100, para los productos de dicha marca la condición será que tengan un stock mayor a 120, y luego se deberá calcular qué porcentaje del total de productos cumple con esta condición.

Para realizar la siguiente consulta vamos a:

1. Importamos el dataset de productos (data/raw/products.csv).
2. Filtramos aquellos que tienen campos invalidos (brand_id, product_id)
3. Mapeamos los campos que nos interesan (brand_id, product_id,) para dejar solo los valores necesarios.
4. Importamos el dataset logs de inventario para calcular el flujo de los productos (data/raw/inventory_logs.csv).
5. Filtramos aquellas entradas que tengan campos relevantes invalidos (product_id, quantity).
6. Mapeamos los campos para dejar solo los valores necesarios (product_id, quantity). Debemos acomodar quantity para que sea positivo o negativo dependiendo del tipo de movimiento.
7. Agrupamos por producto para calcular el flujo de los mismos.
8. Filtramos los que tengan un flujo valido (mayor a 0).
9. Joineamos los datasets por "product_id" para quedarnos solo con los productos su flujo.
10. Aplicamos un cache para optimizar la consulta.
11. Agrupamos por marca para calcular el stock stock promedio por marca.
12. Retomamos el rdd cacheado y lo joineamos con el rdd de métricas por marca.
13. Filtramos aquellos productos cuyo stock es al menos 20% más alto que el stock promedio de su marca.
14. Aplicamos un count para obtener la cantidad de productos que cumplen la condición.
15. Aplicamos un count para obtener la cantidad total de productos al rdd cacheado.
16. Calculamos el porcentaje.

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Query5_ProductosMuchoStock").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [6]:
products_df = spark.read.csv(
    "../../data/raw/products.csv", header=True, inferSchema=True
)
products_rdd = products_df.rdd

products_min = products_rdd.filter(
    lambda r: getattr(r, "product_id", None) is not None
    and getattr(r, "brand", None) is not None
).map(
    lambda r: {
        "product_id": str(getattr(r, "product_id", "")).strip(),
        "brand": str(getattr(r, "brand", "")).strip().title(),
    }
)


def normalize_from_set(val, allowed_set):
    if val is None:
        return None
    raw = str(val).strip().upper()
    return raw if raw in allowed_set else None


MOVEMENT_SIGN = {"IN": 1, "OUT": -1, "ADJUSTMENT": 1}

inv_df = spark.read.csv(
    "../../data/raw/inventory_logs.csv", header=True, inferSchema=True
)
inv_rdd = inv_df.rdd

inv_cleaned = (
    inv_rdd.filter(
        lambda r: getattr(r, "product_id", None) is not None
        and getattr(r, "movement_type", None) is not None
        and getattr(r, "quantity_change", None) is not None
    )
    .map(
        lambda r: {
            "product_id": str(getattr(r, "product_id", "")).strip(),
            "movement_type": normalize_from_set(
                getattr(r, "movement_type", None), MOVEMENT_SIGN.keys()
            ),
            "quantity_change": int(getattr(r, "quantity_change", 0) or 0),
        }
    )
    .filter(lambda x: x["movement_type"] is not None)
    .map(
        lambda x: {
            "product_id": x["product_id"],
            "qty": x["quantity_change"] * MOVEMENT_SIGN[x["movement_type"]],
        }
    )
)

flow_by_product = inv_cleaned.map(lambda x: (x["product_id"], x["qty"])).reduceByKey(
    lambda a, b: a + b
)

stock_by_product = flow_by_product.filter(lambda kv: kv[1] >= 0)

In [7]:
prod_kv = products_min.map(lambda x: (x["product_id"], x["brand"]))
joined = stock_by_product.join(prod_kv).cache()  # (product_id, (stock, brand))

by_brand_stock = joined.map(lambda kv: (kv[1][1], (kv[1][0], 1)))
brand_totals = by_brand_stock.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))
brand_avg = brand_totals.mapValues(lambda s: s[0] / s[1] if s[1] else 0.0)  # (brand, avg_stock)

products_by_brand = joined.map(lambda kv: (kv[1][1], (kv[0], kv[1][0])))  # (brand, (product_id, stock))
brand_join = products_by_brand.join(brand_avg)  # (brand, ((product_id, stock), avg_stock))

THRESHOLD = 1.2
qualifying = brand_join.filter(lambda kv: kv[1][0][1] >= THRESHOLD * kv[1][1])

num_qualifying = qualifying.count()
num_total = joined.count()
percentage = (num_qualifying / num_total * 100.0) if num_total > 0 else 0.0

In [10]:
print("Productos con stock >= 120% del promedio de su marca")

print(f"Cantidad: {num_qualifying} de un total de {num_total} productos con stock")
print(f"Porcentaje: {percentage:.2f}%")

Productos con stock >= 120% del promedio de su marca
Cantidad: 28256 de un total de 84869 productos con stock
Porcentaje: 33.29%


25/10/06 03:48:46 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:131)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:700)
	at org.apache.spark.storage.BlockManagerMasterE